**LangChain**


In [ ]:
#Установим необходимые библиотеки:
!pip install -qU langchain
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain-community
!pip install -qU wikipedia

# или !pip install -qU langchain langchain-openai wikipedia

https://platform.openai.com/account/api-keys

https://smith.langchain.com/

https://www.youtube.com/watch?v=VOwI5RsPyzc - Как использовать прокси в браузере

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# опционально:
# os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")





In [ ]:
import os
print("OPENAI_API_KEY:", bool(os.getenv("OPENAI_API_KEY")))
print("LANGCHAIN_API_KEY:", bool(os.getenv("LANGCHAIN_API_KEY")))


In [ ]:
from langchain_openai import ChatOpenAI


In [ ]:
model = ChatOpenAI(
	model="gpt-4o-mini",
	temperature=0.7,
	max_tokens=1000
	)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
SystemMessage(content="Translate the following from English into Italian"),
HumanMessage(content="hi!"),

]

model.invoke(messages)

Используем конструкцию, чтобы получить исключительно ответ от языковой модели.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

In [ ]:
chain = model | parser
chain.invoke(messages)

Проверим LangSmith!

In [ ]:
#получим стоимость запроса
from langchain_community.callbacks.manager import get_openai_callback

with get_openai_callback() as cb:
    result = chain.invoke(messages)
    print(cb)

In [ ]:
#проверим другую модель
model4 = ChatOpenAI(model="gpt-4o")
chain4 = model4 | parser

with get_openai_callback() as cb:
    result = chain4.invoke(messages)
    print(cb)

In [ ]:
print(f"Total Cost (USD): {cb.total_cost}$")

In [ ]:
#Используем шаблон промпта
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

In [ ]:
#доступ к сообщениям
result.to_messages()#[0]

In [ ]:
result.to_messages()[0]

In [ ]:
#добавляем шаблон
chain = prompt_template | model | parser

In [ ]:
chain.invoke({"language": "italian", "text": "hi"})

In [ ]:
#Пример использования
system_template = "Ты - полезный бот массовик затейник. Ты знаешь много способов провести интересно время. Тебя зовут {name}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
    ("user", "Меня зовут Михаил. Я люблю карточные игры, особенно покер. Хочу научиться играть в преферанс. \
        Не люблю гулять на свежем воздухе"),

        ("user", "{user_input}")]

)

result = prompt_template.invoke({"name": "Бодрый друг", "user_input": "На улице солнечно. Чем посоветуешь заняться?"})

chain = prompt_template | model | parser
data = {"name": "Бодрый друг", "user_input": "На улице солнечно. Чем посоветуешь заняться?"}
chain.invoke(data)

In [ ]:
#ответ в режиме стрима
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    )
for chunk in chat.stream("Напиши тезисы про важность промпт-инжиниринга"):
    print(chunk.content, end="", flush=True)

 LangChain. Взаимодействие с языковыми моделями: Часть 2

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# Pydantic-модель
class Person(BaseModel):
    name: str = Field(description="Полное имя человека")
    age: int = Field(description="Возраст человека")
    hobbies: List[str] = Field(description="Список хобби")

# Парсер
parser = PydanticOutputParser(pydantic_object=Person)

# Шаблон промпта
prompt = PromptTemplate(
    template=(
        "Сгенерируй информацию о человеке.\n"
        "{format_instructions}\n"
        "Описание: {input}"
    ),
    input_variables=["input"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | model | parser

result = chain.invoke({"input": "Молодая женщина, увлекается бегом и йогой"})
print(result)
